# last.fm Data Exploraton - Top Artists

In [4]:
# Defining API KEY as variable
API_KEY = '2357e38959224b0d01d286b5993f972c'

# Defining USER_AGENT as a variable
USER_AGENT = 'recatoma'

# Defining Shared Secred as a variable
SHARED_SECRET = '52a457c8fd0c979cd8d267188d7e0523'


In [5]:
# Importing requests package

import requests

In [6]:
#  Specifying user-agent header to idenfity user
headers = {
    'user-agent': USER_AGENT
}

# Specifying the API key and table we want to get from the API
payload = {
    'api_key': API_KEY,
    'method': 'user.getTopArtists',
    'user' : 'recatoma',
    'format': 'json'
}

In [7]:
# Requesting API data
r = requests.get('http://ws.audioscrobbler.com/2.0/', headers=headers, params=payload)

# Return status of the request
r.status_code

200

In [8]:
# Defining function to get the API data 

def lastfm_get(payload):
    # Define headers and URL
    headers = {'user-agent': USER_AGENT}
    url = 'http://ws.audioscrobbler.com/2.0/'

    # Add API key and format to the payload
    payload['api_key'] = API_KEY
    payload['format'] = 'json'
    payload['user'] = 'recatoma'
    payload['limit'] = 50
    response = requests.get(url, headers=headers, params=payload)
    return response

In [9]:
# Running the function to get the Top Artists table
r = lastfm_get({
    'method': 'user.getTopArtists'
})

r.status_code

200

In [10]:
# Importing json module
import json

# Creating function to print response from the API
def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

jprint(r.json())

{
    "topartists": {
        "@attr": {
            "page": "1",
            "perPage": "50",
            "total": "922",
            "totalPages": "19",
            "user": "recatoma"
        },
        "artist": [
            {
                "@attr": {
                    "rank": "1"
                },
                "image": [
                    {
                        "#text": "https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png",
                        "size": "small"
                    },
                    {
                        "#text": "https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png",
                        "size": "medium"
                    },
                    {
                        "#text": "https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821562f.png",
                        "size": "large"
                    },
                    {
                        "#text": "https:

In [11]:
# Checking the first dictionary of the table

jprint(r.json()['topartists']['@attr'])

{
    "page": "1",
    "perPage": "50",
    "total": "922",
    "totalPages": "19",
    "user": "recatoma"
}


In [12]:
# Importing requests-cache library
import requests_cache

# Function to cache the API requests
requests_cache.install_cache()

In [13]:
# Redefining function to get the API data 
def lastfm_get(payload):
    # Define headers and URL
    headers = {'user-agent': USER_AGENT}
    url = 'http://ws.audioscrobbler.com/2.0/'

    # Add API key and format to the payload
    payload['api_key'] = API_KEY
    payload['format'] = 'json'

    response = requests.get(url, headers=headers, params=payload)
    return response

In [14]:
import time
from IPython.core.display import clear_output

# Empty list to store the API content
responses = []

page = 1
total_pages = 99999 # This is just a dummy number so the loop starts

while page <= total_pages:
    payload = {
        'method': 'user.getTopArtists',
        'user' : 'recatoma',
        'limit': 50,
        'page': page
    }
    # Print some output so we can see the status
    print("Requesting page {}/{}".format(page, total_pages))
    # Clear the output to make things neater
    clear_output(wait = True)

    # Make the API call
    response = lastfm_get(payload)

    # If we get an error, print the response and halt the loop
    if response.status_code != 200:
        print(response.text)
        break

    # Extract pagination info
    page = int(response.json()['topartists']['@attr']['page'])
    total_pages = int(response.json()['topartists']['@attr']['totalPages'])

    # Append response
    responses.append(response)

    # If it's not a cached result, sleep
    if not getattr(response, 'from_cache', False):
        time.sleep(0.25)

    # Increment the page number
    page += 1

Requesting page 19/19


In [15]:
# Importing pandas library
import pandas as pd

# Extracting the data from the responses list
r0 = responses[0]

# Converting the json file to a dictionary
r0_json = r0.json()

# Selecting the desired subdictionary
r0_artists = r0_json['topartists']['artist']

# Converting the dictionary to a DataFrame
r0_df = pd.DataFrame(r0_artists)

# Visualizng the new DataFrame
r0_df.head()

,@attr,mbid,url,playcount,image,name,streamable
0,{'rank': '1'},f07dbc2f-317b-470f-bad4-5f1b0eb6faf1,https://www.last.fm/music/Caetano+Veloso,189,"[{'size': 'small', '#text': 'https://lastfm.fr...",Caetano Veloso,0
1,{'rank': '2'},5441c29d-3602-4898-b1a1-b77fa23b8e50,https://www.last.fm/music/David+Bowie,131,"[{'size': 'small', '#text': 'https://lastfm.fr...",David Bowie,0
2,{'rank': '3'},f85710a3-3ef2-48a2-ac20-a87203dd478c,https://www.last.fm/music/Watson,122,"[{'size': 'small', '#text': 'https://lastfm.fr...",Watson,0
3,{'rank': '4'},21042d7e-4622-45f0-b1a0-a6dfa458a2e3,https://www.last.fm/music/Kinky,102,"[{'size': 'small', '#text': 'https://lastfm.fr...",Kinky,0
4,{'rank': '5'},a670e05a-cea8-4b37-bce9-d82daf1a0fa4,https://www.last.fm/music/Feist,99,"[{'size': 'small', '#text': 'https://lastfm.fr...",Feist,0


Another way of build the DataFrame:

In [16]:
# List comprehension
frames = [pd.DataFrame(r.json()['topartists']['artist']) for r in responses]

# Creating the DataFrame by using the concatenate method
recatoma_artists = pd.concat(frames)

# Visualizing the DataFrame
recatoma_artists.head()

,@attr,mbid,url,playcount,image,name,streamable
0,{'rank': '1'},f07dbc2f-317b-470f-bad4-5f1b0eb6faf1,https://www.last.fm/music/Caetano+Veloso,189,"[{'size': 'small', '#text': 'https://lastfm.fr...",Caetano Veloso,0
1,{'rank': '2'},5441c29d-3602-4898-b1a1-b77fa23b8e50,https://www.last.fm/music/David+Bowie,131,"[{'size': 'small', '#text': 'https://lastfm.fr...",David Bowie,0
2,{'rank': '3'},f85710a3-3ef2-48a2-ac20-a87203dd478c,https://www.last.fm/music/Watson,122,"[{'size': 'small', '#text': 'https://lastfm.fr...",Watson,0
3,{'rank': '4'},21042d7e-4622-45f0-b1a0-a6dfa458a2e3,https://www.last.fm/music/Kinky,102,"[{'size': 'small', '#text': 'https://lastfm.fr...",Kinky,0
4,{'rank': '5'},a670e05a-cea8-4b37-bce9-d82daf1a0fa4,https://www.last.fm/music/Feist,99,"[{'size': 'small', '#text': 'https://lastfm.fr...",Feist,0


In [17]:
# Dropping columns that won't be used here

recatoma_artists.drop(['@attr', 'mbid', 'url', 'image', 'streamable'], axis =1, inplace = True)

In [18]:
# Stats from the DataFrame

recatoma_artists.info()
recatoma_artists.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 922 entries, 0 to 21
Data columns (total 2 columns):
playcount    922 non-null object
name         922 non-null object
dtypes: object(2)
memory usage: 21.6+ KB


,playcount,name
count,922,922
unique,65,922
top,1,DIIV
freq,411,1


In [19]:
# Function to get all top 5 tags for each artist
def lookup_tags(artist):
    response = lastfm_get({
        'method': 'artist.getTopTags',
        'artist': artist
    })
    
    # If there's an error, return nothing
    if response.status_code != 200:
        return None
    
    # Extract the top 5 tags and turn them into a string
    tags = [t['name'] for t in response.json()['toptags']['tag'][:5]]
    tags_str = ", ".join(tags)
    
    # Rate limiting
    if not getattr(response, 'from_cache', False):
        time.sleep(0.25)
    
    return tags_str

In [20]:
# Testing the function

lookup_tags("The Cure")

'post-punk, new wave, alternative, 80s, rock'

In [21]:
# Using tqdm package to get the artists' tags

from tqdm import tqdm
tqdm.pandas()

# Creating the new column for the tags

recatoma_artists['tags'] = recatoma_artists['name'].progress_apply(lookup_tags)

/home/danielhn/anaconda3/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 922/922 [06:53<00:00,  2.23it/s]


In [25]:
# Visualizing the DataFrame with the tags info

recatoma_artists.head()

,playcount,name,tags
0,189,Caetano Veloso,"mpb, brazilian, Bossa Nova, tropicalia, brasil"
1,131,David Bowie,"rock, glam rock, classic rock, 80s, alternative"
2,122,Watson,"seen live, Grime, indie, rock, punk"
3,102,Kinky,"rock, alternative, electronic, mexico, latin"
4,99,Feist,"female vocalists, indie, singer-songwriter, Ca..."


In [34]:
# Function to get similar artists
def lookup_similar(artist):
    response = lastfm_get({
        'method': 'artist.getSimilar',
        'artist': artist
    })
    
    # If there's an error, return nothing
    if response.status_code != 200:
        return None
    
    # Extract the top 5 tags and turn them into a string
    tags = [t['name'] for t in response.json()['similarartists']['artist'][:10]]
    tags_str = ", ".join(tags)
    
    # Rate limiting
    if not getattr(response, 'from_cache', False):
        time.sleep(0.25)
    
    return tags_str

In [35]:
# Testing the function for similar artists

lookup_similar('The Cure')

'Siouxsie and the Banshees, Joy Division, Bauhaus, New Order, The Glove, The Smiths, Echo & the Bunnymen, Depeche Mode, The Sisters of Mercy, The Chameleons'

In [36]:
# Creating the new column for the tags

recatoma_artists['similar_artists'] = recatoma_artists['name'].progress_apply(lookup_similar)

100%|██████████| 922/922 [12:07<00:00,  1.27it/s]


In [37]:
# Visualizing the DataFrame with the tags info

recatoma_artists.head(10)

,playcount,name,tags,similar_artists
0,189,Caetano Veloso,"mpb, brazilian, Bossa Nova, tropicalia, brasil","Gilberto Gil, Gal Costa, Maria Bethânia, Chico..."
1,131,David Bowie,"rock, glam rock, classic rock, 80s, alternative","Tin Machine, Lou Reed, Iggy Pop, Roxy Music, T..."
2,122,Watson,"seen live, Grime, indie, rock, punk","Club Silêncio, J-Sweet, Blazin Beats, Kosha, R..."
3,102,Kinky,"rock, alternative, electronic, mexico, latin","Plastilina Mosh, Zoé, Jumbo, La Gusana Ciega, ..."
4,99,Feist,"female vocalists, indie, singer-songwriter, Ca...","Cat Power, Broken Social Scene, Laura Marling,..."
5,93,Björk,"electronic, alternative, experimental, female ...","The Sugarcubes, Kate Bush, FKA twigs, Arca, Gr..."
6,91,Wilco,"Alt-country, indie, seen live, indie rock, alt...","Jeff Tweedy, Tweedy, Uncle Tupelo, Loose Fur, ..."
7,87,Stevie Wonder,"soul, funk, motown, pop, rnb","Marvin Gaye, The Isley Brothers, Earth, Wind &..."
8,87,The Flaming Lips,"indie, seen live, alternative, indie rock, rock","Animal Collective, Ween, Mercury Rev, of Montr..."
9,81,The Rolling Stones,"classic rock, rock, british, 60s, blues","The Who, Keith Richards, Led Zeppelin, The Kin..."


In [38]:
# Exporting the DataFrame to csv

recatoma_artists.to_csv('recatoma_artists.csv', index = None, header=True)